# Методы переменной метрики
Реализуйте 3 варианта методов переменной метрики: одноранговую коррекцию, DFP и BFGS.

In [ ]:
import numpy as np

def line_search(f, df, x, h, alpha=0.1, beta=0.8, gamma=0.9, max_iters=3):
    t = 1.0
    f_x = f(x)
    df_x = df(x)

    for _ in range(max_iters):
        if f(x + h * t) - f_x <= alpha * np.dot(df_x, t * h):
            break
        if np.dot(df(x + t * h), t * h) < gamma * np.dot(df_x, t * h):
            break
        t *= beta
    return t

def rank_correction(f,
                    df,
                    x_0,
                    iters=100):
    """
    Решает задачу оптимизации методом одноранговой коррекции
    Args:
        f: ndarray(n) -> float -- целевая функция
        df: ndarray(n) -> ndarray(n) -- градиент целевой функции
        x_0: ndarray(n) -- начальное приближение
        iters: int -- количество итераций
        
    Returns:
        [x_0, ..., x_iters] -- промежуточные приближения метода
    """
    pass

In [ ]:
def DFP(f,
        df,
        x_0,
        iters=100):
    """
    Решает задачу оптимизации методом DFP
    Args:
        f: ndarray(n) -> float -- целевая функция
        df: ndarray(n) -> ndarray(n) -- градиент целевой функции
        x_0: ndarray(n) -- начальное приближение
        iters: int -- количество итераций
        
    Returns:
        [x_0, ..., x_iters] -- промежуточные приближения метода
    """
    pass

In [ ]:
def BFGS(f,
        df,
        x_0,
        iters=100):
    """
    Решает задачу оптимизации методом BFGS
    Args:
        f: ndarray(n) -> float -- целевая функция
        df: ndarray(n) -> ndarray(n) -- градиент целевой функции
        x_0: ndarray(n) -- начальное приближение
        iters: int -- количество итераций
        
    Returns:
        [x_0, ..., x_iters] -- промежуточные приближения метода
    """
    pass

In [ ]:
def sigmoid(t):
    return 1 / (1 + np.exp(-t))

def cross_entropy(X, y, theta):
    predictions = sigmoid(X @ theta)
    return -1 / X.shape[0] * np.sum(y * predictions + (1 - y) * (1 - predictions))

def cross_entropy_grad(X, y, theta):
    return -(1 / X.shape[0]) * X.T @ (y - sigmoid(X @ theta))

def logistic_regression_gd(X, Y, alpha, batch_size, epochs):
    theta = np.zeros(X.shape[1])
    result = [theta.copy()]
    for _ in range(epochs):
        for i in range(X.shape[0] // batch_size):
            x = X[i * batch_size:(i+1) * batch_size,:]
            y = Y[i * batch_size:(i+1) * batch_size]
            theta = theta + alpha * (1 / x.shape[0]) * x.T @ (y - sigmoid(x @ theta))
        result.append(theta.copy())
    return result

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (7, 7)

In [ ]:
def show_logistic_regression(batch_size):
    n = 200
    m = 1000
    x = np.concatenate((np.random.rand(m, n) - 1, np.ones((m, 1))), axis=1)
    a = np.random.rand(n + 1)
    b = 0.2

    y = (np.sign(x @ a + np.ones(m) * b) + 1) / 2
    alpha = 0.2
    iters = 40
    
    trajectory_sgd = logistic_regression_gd(x, y, alpha, batch_size, iters)
    trajectory_src = rank_correction(lambda theta: cross_entropy(x, y, theta),
                                     lambda theta: cross_entropy_grad(x, y, theta),
                                     np.zeros(x.shape[1]),
                                     iters=iters)
    trajectory_dfp = DFP(lambda theta: cross_entropy(x, y, theta),
                                     lambda theta: cross_entropy_grad(x, y, theta),
                                     np.zeros(x.shape[1]),
                                     iters=iters)
    trajectory_bfgs = BFGS(lambda theta: cross_entropy(x, y, theta),
                                     lambda theta: cross_entropy_grad(x, y, theta),
                                     np.zeros(x.shape[1]),
                                     iters=iters)  
    #plt.yscale('log')
    q = [i for i in range(iters + 1)]
    #print(trajectory_bfgs)
    
    
    f_min = min(min([cross_entropy(x, y, theta) for theta in trajectory_sgd]),
                min([cross_entropy(x, y, theta) for theta in trajectory_src]),
                min([cross_entropy(x, y, theta) for theta in trajectory_dfp]),
                min([cross_entropy(x, y, theta) for theta in trajectory_bfgs]))
    plt.plot(q, [cross_entropy(x, y, theta)-f_min for theta in trajectory_sgd], label='SGD')
    plt.plot(q, [cross_entropy(x, y, theta)-f_min for theta in trajectory_src], label='Rank-1')
    plt.plot(q, [cross_entropy(x, y, theta)-f_min for theta in trajectory_bfgs], label='BFGS')
    plt.plot(q, [cross_entropy(x, y, theta)-f_min for theta in trajectory_dfp], label='DFP')

    
    plt.yscale('log')
    plt.legend()
    plt.show()

In [ ]:
show_logistic_regression(1)